In [ ]:
#This script cleans the data based on EDA done in a seperate script.

In [1]:
# import needed libraries.
import pandas as pd
import geopandas as gpd # geographic data manipulation
import numpy as np

import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation

#this is for the heatmap.
import folium
from folium.plugins import HeatMap

from google.colab import drive #to mount google drive to the vm
import shutil

import os
!pip install python-dotenv
from dotenv import load_dotenv

# Load the .env file
load_dotenv('/content/.env')
# Mount Google Drive (need to be run only once! and is valid when working on Google Colab)
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# download the dataset from google drive.
# Download the file from Google Drive using the file ID
!gdown {os.getenv('full-subway-manhattan-data')}

Downloading...
From (original): https://drive.google.com/uc?id=1BPFJv4QlUPr3Vywvk5dluiq9DQV19mhm
From (redirected): https://drive.google.com/uc?id=1BPFJv4QlUPr3Vywvk5dluiq9DQV19mhm&confirm=t&uuid=35eedcfa-add8-47e3-811d-6a04cb2e2eb2
To: /content/subway-data-may-29.csv
100% 9.95G/9.95G [01:50<00:00, 90.2MB/s]


In [6]:
#load the dataset into pandas df
df = pd.read_csv(f"/content/subway-data-may-29.csv")

<ipython-input-6-dde7986a472e>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"/content/subway-data-may-29.csv")


In [7]:
#show a sample of the data
df.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,04/25/2024 06:00:00 AM,subway,616,"74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,...",Queens,metrocard,Metrocard - Unlimited 7-Day,411,0,40.746850,-73.891335,POINT (-73.891335 40.74685)
1,04/25/2024 08:00:00 AM,subway,419,"225 St (2,5)",Bronx,metrocard,Metrocard - Fair Fare,14,1,40.888023,-73.860344,POINT (-73.860344 40.888023)
2,04/25/2024 01:00:00 PM,subway,133,Atlantic Av (L),Brooklyn,metrocard,Metrocard - Other,2,0,40.675346,-73.903100,POINT (-73.9031 40.675346)
3,04/25/2024 02:00:00 PM,subway,360,Pelham Bay Park (6),Bronx,metrocard,Metrocard - Fair Fare,16,3,40.852463,-73.828120,POINT (-73.82812 40.852463)
4,04/25/2024 05:00:00 PM,subway,222,Roosevelt Island (F),Manhattan,metrocard,Metrocard - Seniors & Disability,21,0,40.759144,-73.953260,POINT (-73.95326 40.759144)


In [30]:

# check the earliest and latest time of the data
print(df['transit_timestamp'].min())
print(df['transit_timestamp'].max())

2022-02-01 00:00:00
2024-05-23 23:00:00


In [8]:
#number of rows and cols
df.shape

(58306773, 12)

In [9]:
#check the data types
df.dtypes

transit_timestamp       object
transit_mode            object
station_complex_id      object
station_complex         object
borough                 object
payment_method          object
fare_class_category     object
ridership                int64
transfers                int64
latitude               float64
longitude              float64
Georeference            object
dtype: object

In [10]:
#show the number of unique value in each column.
df.nunique()

transit_timestamp      20227
transit_mode               3
station_complex_id       853
station_complex          428
borough                    5
payment_method             2
fare_class_category       10
ridership               7175
transfers               1042
latitude                 926
longitude                927
Georeference             976
dtype: int64

In [11]:
#drop rows for which the borough is not Manhattan
# Filter rows where borough is not Manhattan (keep rows that have borough == manhattan)
df = df[df['borough'] == 'Manhattan']
df.drop(columns=['borough'], inplace=True)

# Reset index to have consecutive indices
df.reset_index(drop=True, inplace=True)

<ipython-input-11-d886e1c8d1e3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['borough'], inplace=True)


In [12]:
#drop the "payment_method" column as its content can be derived from the "fare_class_category" column.
df.drop(columns=['payment_method'], inplace=True)

# Reset index to have consecutive indices
df.reset_index(drop=True, inplace=True)

In [13]:
#drop the "transit_mode" column as it is mostly a constant column, and the transfer mood is not important for the analysis.
df.drop(columns=['transit_mode'], inplace=True)

# Reset index to have consecutive indices
df.reset_index(drop=True, inplace=True)

In [14]:
#drop the "Georeference" column as its content will be reconstructed using "latitude" and "longitude" columns .
df.drop(columns=['Georeference'], inplace=True)

# Reset index to have consecutive indices
df.reset_index(drop=True, inplace=True)

In [15]:
#ensure that the precision of the long and lat is about 10 metre by rounding to 4 decimal points.
df['latitude'] = df['latitude'].round(4)
df['longitude'] = df['longitude'].round(4)

#reconstruct georeference from the long and lat cols.
df['georeference'] = df.apply(lambda row: f"POINT ({row['longitude']} {row['latitude']})", axis=1)

In [16]:
# Make sure the "station_complex_id" col only has numerical values.
# Replace 'TRAM1' with 888 and 'TRAM2' with 999 in the "station_complex_id" col
df['station_complex_id'] = df['station_complex_id'].replace({'TRAM1': 888, 'TRAM2': 999})
# Convert the 'id' column to integers
df['station_complex_id'] = df['station_complex_id'].astype(int)

In [17]:
#check number of rows and cols after all those modification above
#check number of columns after deletion
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns now")

There are 17806410 rows and 9 columns now


In [19]:
# Convert transit_timestamp to datetime.
df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])

# Convert categorical columns to category data type
categorical_columns = ['station_complex_id', 'station_complex', 'fare_class_category']
df[categorical_columns] = df[categorical_columns].astype('category')

In [20]:
#check the new types now
df.dtypes

transit_timestamp      datetime64[ns]
station_complex_id           category
station_complex              category
fare_class_category          category
ridership                       int64
transfers                       int64
latitude                      float64
longitude                     float64
georeference                   object
dtype: object

In [21]:
# check for Null, N/A and missing values by column
df.isnull().sum()

transit_timestamp      0
station_complex_id     0
station_complex        0
fare_class_category    0
ridership              0
transfers              0
latitude               0
longitude              0
georeference           0
dtype: int64

In [22]:
#number of rows before dropping duplicate values
df.count()

transit_timestamp      17806410
station_complex_id     17806410
station_complex        17806410
fare_class_category    17806410
ridership              17806410
transfers              17806410
latitude               17806410
longitude              17806410
georeference           17806410
dtype: int64

In [23]:
#number of rows after dropping duplicate values (we drop the duplicate values because they are redundant.)
df = df.drop_duplicates()

df.count()

transit_timestamp      17806410
station_complex_id     17806410
station_complex        17806410
fare_class_category    17806410
ridership              17806410
transfers              17806410
latitude               17806410
longitude              17806410
georeference           17806410
dtype: int64

In [24]:
# Count the number of records for each hour
df_hour = df.copy()
df_hour['hour'] = df_hour['transit_timestamp'].dt.hour
hourly_counts = df_hour['hour'].value_counts().sort_index()

# Display the hourly counts
hourly_counts

hour
0     626091
1     522599
2     452437
3     434423
4     513210
5     651162
6     757269
7     806761
8     824028
9     826805
10    829664
11    836477
12    844165
13    846066
14    848285
15    849780
16    851449
17    851444
18    844579
19    829077
20    813029
21    743621
22    719059
23    684930
Name: count, dtype: int64

In [25]:
# Filter rows where transfers > ridership [this should never be the case as transfers are subset of ridership!]
condition = df['transfers'] > df['ridership']
filtered_df = df[condition]

# Get the count of such rows
count = filtered_df.shape[0]

print("Count of rows where transfers > ridership:", count)

Count of rows where transfers > ridership: 0


In [26]:
# Get the number of unique values in the 'station_complex_id', 'station_complex', 'georeference' columns
count_unique_ids = df['station_complex_id'].nunique()
print(f'Number of unique station ids: {count_unique_ids}')

count_unique_stations = df['station_complex'].nunique()
print(f'Number of unique stations: {count_unique_stations}')

count_unique_geos = df['georeference'].nunique()
print(f'Number of unique georeferences: {count_unique_geos}')

Number of unique station ids: 123
Number of unique stations: 123
Number of unique georeferences: 168


In [27]:
#drop the "georeference" column again as its content will be reconstructed using "latitude" and "longitude" columns in a geodataframe when doing the analysis later.
df.drop(columns=['georeference'], inplace=True)

# Reset index to have consecutive indices
df.reset_index(drop=True, inplace=True)


In [29]:
df.to_csv("cleaned-subway-data.csv")

In [31]:
# Export to Gooogle Drive
try:
    # Specify the path of the file in Colab VM
    file_path_in_colab = f'/content/cleaned-subway-data.csv'

    # Define the destination folder in Google Drive
    destination_folder = '/content/drive/My Drive/Colab Notebooks/'

    # Copy the file to the destination folder in Google Drive
    shutil.copy(file_path_in_colab, destination_folder + f'cleaned-subway-data.csv')
    print(f"CSV file has been successfully uploaded to Google Drive.")

except Exception as e:
    print(f"An error occurred: {e}")

CSV file has been successfully uploaded to Google Drive.
